<a href="https://colab.research.google.com/github/enVives/Caltech101/blob/main/Caltech101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [109]:
from itertools import filterfalse
import torch,torchvision
import numpy as np
import pandas as pd
import cv2
import time
import os

from torchvision import transforms,models
from torch import nn
from IPython.display import clear_output,display
from torch.utils.data import DataLoader, random_split,Subset,Dataset
from google.colab.patches import cv2_imshow
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from glob import glob

ROOT = './sample_data/'
CLASS_A = 'cougar_body'
CLASS_B = 'windsor_chair'

DOWNLOAD = False
#!rm -rf /content/sample_data/*
# torch.cuda.default_stream(torch.device('cuda'))

In [110]:
def find_mean_std(dataset):
  dataloader = DataLoader(dataset,batch_size = 1,shuffle=FALSE)
  mean = torch.zeros(3)
  std = torch.zeros(3)
  samples = 0

  for image,_ in dataloader:
    mean += image.mean(dim=[0, 2, 3])  # Mean for each channel [batchsize,channels,height,width]
    std += image.std(dim=[0, 2, 3])    # Std for each channel
    samples += 1

  mean /= samples
  std /= samples

  print(mean)
  print(std)

In [111]:
class Formes(Dataset):
    def __init__(self, paths, labels= None, transforms = None):
        self.images = paths
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return len(self.labels)

    def __getdist__(self):
      return pd.Series(self.labels).value_counts()

    def __getitem__(self, index):
      path = self.images[index]
      label = self.labels[index]

      image = cv2.imread(path, cv2.IMREAD_COLOR)
      image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
      image = self.transforms(image)

      return image, label

In [112]:
def get_labels(sett):
  labels = np.array([])
  frequency = np.zeros(101)
  for _,label in sett:
    labels = np.append(labels,label)
    frequency[label] += 1
  return labels.astype(int),frequency

In [113]:
#from pickle import FALSE
from sklearn.model_selection import train_test_split

TRAINING = 0.75
VAL = 0.15
TESTING = 0.10

#[0.5459, 0.5288, 0.5022]
#[0.2424, 0.2393, 0.2409]
mean = torch.tensor([0.485,0.456,0.406])
std = torch.tensor([0.229,0.224,0.225])

transform = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB") if img.mode != "RGB" else img),  # Ensure all images are RGB
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

transform2 = transforms.Compose([
    transforms.Lambda(lambda img: img.convert("RGB") if img.mode != "RGB" else img),  # Ensure all images are RGB
    transforms.Resize((224, 224)),
    # transforms.RandomHorizontalFlip(p=0.3),
    # transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(degrees=20),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

dataset = torchvision.datasets.Caltech101(root= ROOT,download=DOWNLOAD,transform=transform)
img_files = sorted(glob('/content/sample_data/caltech101/101_ObjectCategories/*/*'))

labels = []
for img_path in img_files:
  label = img_path.split(os.path.sep)[-2]
  labels.append(label)

lb = LabelEncoder()
labels = lb.fit_transform(labels)

weights = 1.0 / pd.Series(labels).value_counts()
weights = weights / weights.sum()
weights = torch.tensor(weights,dtype=torch.float32)

X_train, X_test, y_train, y_test = train_test_split(img_files, labels, test_size=TESTING, random_state=42, stratify=labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=VAL/(TRAINING+VAL), random_state=42, stratify=y_train)

training = Formes(X_train,y_train,transform2)
testing = Formes(X_test,y_test,transform)
validation = Formes(X_val,y_val,transform)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#training,validation,testing = random_split(dataset,[train_size,validation_size,len(dataset)-train_size-validation_size])

In [114]:
print(f"LONGITUD SET DE TRAINING: {training.__len__()}")
#print(f"Distribució training: {training.__getdist__()/training.__len__()}")
with open("training.txt", "w") as file:
    for item in training.__getdist__()/training.__len__():
        file.write(f"{item}\n")
print(f"LONGITUD SET DE VALIDATION: {validation.__len__()}")
#print(f"Distribució validation: {validation.__getdist__()/training.__len__()}")
with open("valid.txt", "w") as file:
    for item in validation.__getdist__()/training.__len__():
        file.write(f"{item}\n")
print(f"LONGITUD SET DE TESTING: {testing.__len__()}")

LONGITUD SET DE TRAINING: 6858
LONGITUD SET DE VALIDATION: 1372
LONGITUD SET DE TESTING: 915


Ara ja sabem que hem de predir si una imatge pertany a les classes 25 o 99

In [115]:
def pick_algorithm(number):
  if number == 0:
    alexnetmulticlass = models.alexnet(weights=None)
    alexnetmulticlass.classifier = nn.Sequential(
    torch.nn.Linear(9216, 1024),
    nn.ReLU(),
    torch.nn.Linear(1024, 1024),
    nn.ReLU(),
    torch.nn.Linear(1024, 512),
    nn.ReLU(),
    torch.nn.Linear(512, 102),  # Ja que tenim 10 classes.
)
    loss_fn = nn.CrossEntropyLoss(weight=weights.to(device))
    return alexnetmulticlass,loss_fn

In [116]:
def veure_imatges(train_data,std,mean):
  for i in range(len(train_data)):
    imatge,label = train_data[i]

    print(imatge.ndimension())
    print(imatge.shape)

    imatge = imatge * (std[:, None, None]*255) + (mean[:, None, None]*255)
    # Convert the tensor back to a NumPy array
    img_numpy = imatge.permute(1, 2, 0).numpy()  # Change from (C, H, W) to (H, W, C)
    cv2_imshow(img_numpy)
    time.sleep(5)
    clear_output(wait=True)

In [117]:
BATCH_SIZE = 32
EPOCHS = 50
PATIENCE = 10

train_loader = torch.utils.data.DataLoader(training, batch_size=BATCH_SIZE, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation, batch_size=BATCH_SIZE, shuffle=True)
testing_loader = torch.utils.data.DataLoader(testing, batch_size=BATCH_SIZE, shuffle=True)

model,loss_fn = pick_algorithm(0)
model.to(device)

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#veure_imatges(training,std,mean)

In [118]:
img, target = next(iter(train_loader))
print(img.shape)

torch.Size([32, 3, 224, 224])


In [119]:
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score
from tqdm.auto import tqdm
import pylab as pl

t_loss = np.zeros(EPOCHS)
v_loss = np.zeros(EPOCHS)
acc_t = np.zeros(EPOCHS) #accuracy
acc_v = np.zeros(EPOCHS)
f1_t = np.zeros(EPOCHS) #f1
f1_v = np.zeros(EPOCHS)
recall_t = np.zeros(EPOCHS) #recall
recall_v = np.zeros(EPOCHS)
precision_t = np.zeros(EPOCHS)
precision_v = np.zeros(EPOCHS) #precisió

epochs_without_improvement = 0
best_val_loss = float('inf')

pbar = tqdm(range(1, EPOCHS + 1))  # tdqm permet tenir text dinàmic

classes = []

def extreu_classes(target):
    for i in target:
      if i not in classes:
        classes.append(i)

for epoch in pbar:

    train_loss = 0
    train_acc = 0
    train_f1 = 0
    train_recall = 0
    train_precision = 0
    val_acc = 0
    val_recall = 0
    val_f1 = 0
    val_loss = 0
    val_precision = 0

    batch_num = 1

    for batch_num, (input_img, target) in tqdm(enumerate(train_loader), desc=f"Batches (Època {epoch})"):

        model.train()

        optimizer.zero_grad()
        #extreu_classes(target.to(device))

        output = model(input_img.to(device))

        # print(target.shape)
        # print(output.shape)

        target = target.to(device)
        loss = loss_fn(output, target)


        model.zero_grad()
        loss.backward()

        with torch.no_grad():
            optimizer.step()

        #print(f"Pèrdua entrenament batch: {batch_num} epoch: {epoch+1}  train_loss: {loss.item()}")
        model.eval()

        y_class_predict = torch.argmax(output, dim=1)
        y_class_predict = (y_class_predict.cpu().detach().numpy())
        target = target.cpu().detach().numpy()


        train_acc += accuracy_score(target,y_class_predict)
        train_f1 += f1_score(target,y_class_predict,zero_division=1,average='macro')
        train_recall += recall_score(target,y_class_predict,zero_division=1,average='macro')
        train_precision += precision_score(target,y_class_predict,zero_division=1,average='macro')
        train_loss += loss.item()

    model.eval()
    with torch.no_grad():
      for batch_num, (input_img, target) in enumerate(validation_loader):


            output = model(input_img.to(device))
            target = target.to(device)
            loss = loss_fn(output, target)

            y_class_predict = torch.argmax(output, dim=1)
            y_class_predict = (y_class_predict.cpu().detach().numpy())

            target = target.cpu().detach().numpy()

            val_acc  += accuracy_score(target,y_class_predict)
            val_f1 += f1_score(target,y_class_predict,zero_division=1,average='macro')
            val_recall += recall_score(target,y_class_predict,zero_division=1,average='macro')
            val_precision += precision_score(target,y_class_predict,zero_division=1,average='macro')

            val_loss += loss.item()

            #print(f"Pèrdua entrenament batch validacio: {batch_num} epoch: {epoch+1}  val_loss: {val_loss.item()}")

            # RESULTATS
    if val_loss < best_val_loss:
      best_val_loss = val_loss
      epochs_without_improvement = 0
    else:
      epochs_without_improvement += 1

    train_loss /= len(train_loader)
    t_loss[epoch - 1] = train_loss

    train_acc /= len(train_loader)
    acc_t[epoch - 1] = train_acc

    train_f1 /= len(train_loader)
    f1_t[epoch - 1] = train_f1

    train_recall /= len(train_loader)
    recall_t[epoch - 1] = train_recall

    train_precision /= len(train_loader)
    precision_t[epoch-1] = train_precision

    print(f"Pèrdua entrenament epoch: {epoch}  train_loss: {train_loss}")
    print(f"Accuracy train epoch: {epoch}  train_acc: {train_acc}")
    print(f"F1 train epoch: {epoch}  train_f1: {train_f1}")
    print(f"Recall train epoch: {epoch}  train_recall: {train_recall}")
    print(f"Precision train epoch: {epoch}  train_recall: {train_precision}")

    val_loss /= len(validation_loader)
    v_loss[epoch - 1] = val_loss

    val_acc /= len(validation_loader)
    acc_v[epoch - 1] = val_acc

    val_f1 /= len(validation_loader)
    f1_v[epoch - 1] = val_f1

    val_recall /= len(validation_loader)
    recall_v[epoch - 1] = val_recall

    val_precision /= len(validation_loader)
    precision_v[epoch-1] = val_precision

    print()
    print()
    print(f"Pèrdua validació epoch: {epoch}  val_loss: {val_loss}")
    print(f"Accuracy val epoch: {epoch}  val_acc: {val_acc}")
    print(f"F1 val epoch: {epoch}  val_f1: {val_f1}")
    print(f"Recall val epoch: {epoch}  val_reall: {val_recall}")
    print(f"Precision val epoch: {epoch}  val_reall: {val_precision}")

    if epochs_without_improvement >= PATIENCE:
        print("Early stopping triggered!")
        break

  0%|          | 0/50 [00:00<?, ?it/s]

Batches (Època 1): 0it [00:00, ?it/s]

Pèrdua entrenament epoch: 1  train_loss: 4.511311635305715
Accuracy train epoch: 1  train_acc: 0.02572674418604651
F1 train epoch: 1  train_f1: 0.0021674952668668734
Recall train epoch: 1  train_recall: 0.04349094751976907
Precision train epoch: 1  train_recall: 0.9563886402801682


Pèrdua validació epoch: 1  val_loss: 4.442874287450036
Accuracy val epoch: 1  val_acc: 0.02616279069767442
F1 val epoch: 1  val_f1: 0.0021290612399447167
Recall val epoch: 1  val_reall: 0.04216185348686418
Precision val epoch: 1  val_reall: 0.9503215232193052


Batches (Època 2): 0it [00:00, ?it/s]

Pèrdua entrenament epoch: 2  train_loss: 4.369281020275382
Accuracy train epoch: 2  train_acc: 0.0770639534883721
F1 train epoch: 2  train_f1: 0.0191658095892853
Recall train epoch: 2  train_recall: 0.09870521614760222
Precision train epoch: 2  train_recall: 0.89902452253217


Pèrdua validació epoch: 2  val_loss: 4.1952469570692195
Accuracy val epoch: 2  val_acc: 0.10465116279069768
F1 val epoch: 2  val_f1: 0.027910614724040817
Recall val epoch: 2  val_reall: 0.11425874817498151
Precision val epoch: 2  val_reall: 0.8751727472209082


...

Pèrdua entrenament epoch: 18  train_loss: 0.34809386501478595
Accuracy train epoch: 18  train_acc: 0.8553779069767442
F1 train epoch: 18  train_f1: 0.7493444562925536
Recall train epoch: 18  train_recall: 0.8793533426649551
Precision train epoch: 18  train_recall: 0.8538822401567517


Pèrdua validació epoch: 18  val_loss: 3.528780274612959
Accuracy val epoch: 18  val_acc: 0.5554401993355482
F1 val epoch: 18  val_f1: 0.3539129618306978
Recall val epoch: 18  val_reall: 0.6540999485689624
Precision val epoch: 18  val_reall: 0.6455670002693377


Batches (Època 19): 0it [00:00, ?it/s]

Pèrdua entrenament epoch: 19  train_loss: 0.2925883655804534
Accuracy train epoch: 19  train_acc: 0.8735174418604651
F1 train epoch: 19  train_f1: 0.7770960706162642
Recall train epoch: 19  train_recall: 0.893902633159192
Precision train epoch: 19  train_recall: 0.869917844825674


Pèrdua validació epoch: 19  val_loss: 3.617890105691067
Accuracy val epoch: 19  val_acc: 0.5491071428571428
F1 val epoch: 19  val_f1: 0.33874301358361003
Recall val epoch: 19  val_reall: 0.6396505004870218
Precision val epoch: 19  val_reall: 0.6399202870305002


Batches (Època 20): 0it [00:00, ?it/s]

Pèrdua entrenament epoch: 20  train_loss: 0.26673525042918533
Accuracy train epoch: 20  train_acc: 0.8790406976744186
F1 train epoch: 20  train_f1: 0.7925951323681749
Recall train epoch: 20  train_recall: 0.899745852581305
Precision train epoch: 20  train_recall: 0.8791153910733922


Pèrdua validació epoch: 20  val_loss: 3.6314215521479762
Accuracy val epoch: 20  val_acc: 0.5419435215946844
F1 val epoch: 20  val_f1: 0.3355457692776793
Recall val epoch: 20  val_reall: 0.6314131558650634
Precision val epoch: 20  val_reall: 0.6292336899146154
Early stopping triggered!
